<a href="https://colab.research.google.com/github/mou-pi-ya/Celebal-Technology-.py/blob/main/Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()  # Upload kaggle.json file


Saving house-prices-advanced-regression-techniques.zip to house-prices-advanced-regression-techniques.zip


{'house-prices-advanced-regression-techniques.zip': b'PK\x03\x04-\x00\x00\x00\x08\x002\xac\x8fO\x91\x9ef\xb9\xff\xff\xff\xff\xff\xff\xff\xff\x14\x00\x14\x00data_description.txt\x01\x00\x10\x00:4\x00\x00\x00\x00\x00\x00^\x11\x00\x00\x00\x00\x00\x00\xed[\xddr\xe3\xb8r\xbe\x86\x9f\x02\x17\xc9D\xaeX;\x96vgg\xc7\xc9\x8d~mUdY#\xdaV\x9c;\x88\x84$\x8eIB\x01Ik|\x9e>_\x03$HJ\xb2g7\xb5\xe7$u\xea\xb8j\x04\x10h4\xd1\xff\xdd \xe6\xd6\xf3\xf2\xd5 \x12iz\xc5\'\x81L\xb2p\x1d\xca\x94g[\xc9\xb3\xd7\x9d\xe4j\xcd\x83\xbd\x8c\xa20\xd9\xf00yQ\xd1\x8b\x0c\xd01\x00\xa9\x88\xe4O\xec\xec\x8c\x17\x7f\xddK\xd6i{\xf7w\x8b\'\xde\xf9\xf2\xcb\xaf\xfc\x03\x9f\x8d\x96\xa3\x05\xefM\xa7\xdc\xbb\x7f\x9a\x8e<\x07\xfas\x03\xf4\x13@\xef\xa6\xc3\xd1\xc2\xcd\xffR\xcd/?\x8e\'\xb3\x89w3\x1a\xf2\xde\xfd\xfdd`\xd0\xf5\xaek\xc8~\xf9\x04\xe0\xce\xc7.\xb7\x0b\xda\xfcaV-9\x04\xfet\xd9\x00~\x1b\xf0\xd7K\xd6=A\x8d\x9b\xff\xdc\x98?&\xe1\xf3\'\xccW/:\xc2\xff\xdb%\xf3\xe6\xd3\xc9=\xbf[\xf0\xdb\x87\xe9\xfd\xa4=\x1d=\x8e\xa6\xd5\xfc\xa7b~|\xf

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle competitions download -c house-prices-advanced-regression-techniques

# Unzip the files
!unzip house-prices-advanced-regression-techniques.zip


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [3]:
import pandas as pd
import numpy as np

# Load train and test data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)


Train shape: (1460, 81)
Test shape: (1459, 80)


In [4]:
# Save target and drop from train
y = train['SalePrice']
train.drop(['SalePrice'], axis=1, inplace=True)

# Combine train and test
combined = pd.concat([train, test], axis=0, ignore_index=True)
print("Combined shape:", combined.shape)


Combined shape: (2919, 80)


In [5]:
# Drop columns with too many missing values
missing = combined.isnull().sum()
drop_cols = missing[missing > 0.4 * combined.shape[0]].index
combined.drop(columns=drop_cols, inplace=True)

# Fill numeric columns with median
num_cols = combined.select_dtypes(include=['int64', 'float64']).columns
combined[num_cols] = combined[num_cols].fillna(combined[num_cols].median())

# Fill categorical columns with mode
cat_cols = combined.select_dtypes(include=['object']).columns
combined[cat_cols] = combined[cat_cols].fillna(combined[cat_cols].mode().iloc[0])


In [6]:
# Example: Total square footage
combined['TotalSF'] = combined['TotalBsmtSF'] + combined['1stFlrSF'] + combined['2ndFlrSF']

# Simplify 'Year' columns
combined['HouseAge'] = 2024 - combined['YearBuilt']
combined['RemodAge'] = 2024 - combined['YearRemodAdd']
combined['GarageAge'] = 2024 - combined['GarageYrBlt'].fillna(0)

# Drop original date columns
combined.drop(columns=['YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], inplace=True)


In [7]:
combined = pd.get_dummies(combined, drop_first=True)
print("Encoded shape:", combined.shape)


Encoded shape: (2919, 231)


In [8]:
X_train = combined.iloc[:len(y), :]
X_test = combined.iloc[len(y):, :]

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)


X_train: (1460, 231)
X_test: (1459, 231)
